In [1]:
import pandas as pd
import numpy as np
import pickle
import random
from tensorflow.keras.models import load_model
from IPython.display import HTML as html_print

In [2]:
# https://github.com/ipavlopoulos/toxic_spans/blob/master/evaluation/semeval2021.py
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [3]:
# https://stackoverflow.com/a/42534887
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

In [4]:
def get_spans(txt):
    if len(txt) > 2:
        return [int(x) for x in txt[1:-1].split(',')]
    else:
        return []


with open('model_data/datafile.pkl','rb') as fp:
    data = pickle.load(fp)

mlen = data['mlen']
alen = data['alen']
alphabet = data['alphabet']

model_name = 'models/Experiment_02_best_mean.h5'
model = load_model(model_name,compile=False)

In [5]:
def get_x(text):
    text = text.lower()
    matx = np.zeros((mlen,alen))
    for i,ch in enumerate(text):
        vx = np.zeros(alen)
        if ch in alphabet:
            ix = alphabet.index(ch)        
            vx[ix] = 1
        matx[i,:] = vx   
    return matx

In [6]:
def print_colored(text,predictions):
    otext = []
    for i in range(len(text)):
        if i in predictions:
            otext.append(cstr(text[i], color='red'))
        else:
            otext.append(cstr(text[i]))
    return html_print(''.join(otext))

In [7]:
thresh = 0.7

In [8]:
df = pd.read_csv('data/tsd_train.csv')

In [9]:
ix = random.randrange(len(df))
text = df.text[ix]
gold = get_spans(df.spans[ix])
x = get_x(text)
out = model.predict(np.expand_dims(x,axis=0))

In [10]:
predictions = [[x for x in range(1056) if pred[x] >= thresh] for pred in out][0]

In [11]:
print('Text : ',text)
print('\nGround truth labels : ',gold)
print('\nPredictions : ',predictions)
print('\nF1 Score : ',f1(predictions,gold))

Text :  It doesn't matter who wins. With a choice between a loudmouthed obnoxious buffoon or a lying cheating thief, Americans are screwed regardless.

Ground truth labels :  [64, 65, 66, 67, 68, 69, 70, 71, 72]

Predictions :  [64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]

F1 Score :  0.6923076923076923


In [12]:
print('Ground truth : ')
print_colored(text,gold)

Ground truth : 


In [13]:
print('Prediction : ')
print_colored(text,predictions)

Prediction : 
